# 🎫 Support Ticket Topic Classification

**Multi-class text classification** to categorize customer support messages into:
- **Billing** — payment, refund, charges, subscription issues
- **Technical** — bugs, crashes, connectivity, device problems
- **Account** — login, password, profile, security issues

**Pipeline:** TF-IDF Vectorization → Logistic Regression

---

## 1. Imports & Setup

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    ConfusionMatrixDisplay,
)

# Display settings
pd.set_option("display.max_colwidth", 120)
sns.set_theme(style="whitegrid", palette="muted")

print("✅ All imports loaded successfully.")

## 2. Configuration

In [ ]:
# Path to the full Twitter Customer Support dataset (optional)
DATA_PATH = "twcs/twcs.csv"

# How many rows to sample from the full dataset (for speed)
SAMPLE_SIZE = 50_000

# Train/test split ratio
TEST_SIZE = 0.2

# Random seed for reproducibility
RANDOM_STATE = 42

# Category keyword dictionaries for labeling
CATEGORY_KEYWORDS = {
    "Technical": [
        "bug", "crash", "error", "slow", "update", "wifi", "battery",
        "freeze", "install", "broken", "fix", "glitch", "lag",
        "disconnect", "loading", "device", "software", "hardware",
        "printer", "app", "ios", "android", "bluetooth", "screen",
        "reboot", "restart", "download", "sync", "connection",
    ],
    "Billing": [
        "bill", "charge", "refund", "payment", "price", "invoice",
        "subscription", "fee", "credit", "cost", "pay", "money",
        "overcharg", "renew", "cancel", "plan", "pricing", "receipt",
        "discount", "coupon",
    ],
    "Account": [
        "account", "password", "login", "sign in", "profile",
        "username", "email", "register", "verify", "reset",
        "locked", "access", "dm", "log in", "sign up", "settings",
        "security", "two factor", "authentication",
    ],
}

print("✅ Configuration set.")
print(f"   Categories: {list(CATEGORY_KEYWORDS.keys())}")
print(f"   Total keywords: {sum(len(v) for v in CATEGORY_KEYWORDS.values())}")

## 3. Helper Functions

In [ ]:
def preprocess_text(text: str) -> str:
    """Clean a raw message for classification.

    Steps:
      1. Lowercase
      2. Remove URLs
      3. Remove @mentions
      4. Remove hash symbols
      5. Keep only letters and spaces
      6. Collapse whitespace
    """
    text = str(text).lower()
    text = re.sub(r"http\S+|www\.\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def assign_label(text: str):
    """Assign a category label based on keyword matching.

    Priority order: Technical > Billing > Account.
    Returns None if no keywords match.
    """
    cleaned = preprocess_text(text)
    for category, keywords in CATEGORY_KEYWORDS.items():
        if any(kw in cleaned for kw in keywords):
            return category
    return None


print("✅ Helper functions defined.")

### 3.1 Test Preprocessing

In [ ]:
# ── Unit tests for preprocess_text ──
test_cases = [
    ("Hello @user https://example.com #support", "hello support"),
    ("My APP keeps CRASHING!!!", "my app keeps crashing"),
    ("Can't   log   in  ", "cant log in"),
    ("I was charged $50 twice", "i was charged  twice"),
    ("", ""),
]

print("Testing preprocess_text():")
all_passed = True
for raw, expected in test_cases:
    result = preprocess_text(raw)
    status = "✅" if result == expected else "❌"
    if result != expected:
        all_passed = False
    print(f"  {status}  {raw!r:50s} → {result!r}")
    if result != expected:
        print(f"       Expected: {expected!r}")

print()
print("All preprocessing tests passed! ✅" if all_passed else "Some tests FAILED ❌")

In [ ]:
# ── Unit tests for assign_label ──
label_test_cases = [
    ("I was charged twice on my credit card", "Billing"),
    ("My app keeps crashing after the update", "Technical"),
    ("I can't log into my account", "Account"),
    ("The wifi is not working at all", "Technical"),
    ("I need a refund for the subscription", "Billing"),
    ("How do I reset my password?", "Account"),
    ("Hello there, good morning", None),  # No keywords → None
]

print("Testing assign_label():")
all_passed = True
for text, expected in label_test_cases:
    result = assign_label(text)
    status = "✅" if result == expected else "❌"
    if result != expected:
        all_passed = False
    print(f"  {status}  {text:50s} → {result} (expected {expected})")

print()
print("All labeling tests passed! ✅" if all_passed else "Some tests FAILED ❌")

## 4. Load & Prepare Data

This section tries to load the full `twcs.csv` dataset. If the file is not found, it falls back to a **built-in sample dataset** (60 labeled messages) so the notebook is always runnable.

In [ ]:
def create_sample_dataset() -> pd.DataFrame:
    """Create a built-in sample dataset for testing without external files."""
    data = [
        # ── Technical (20 samples) ──
        {"text": "My app keeps crashing every time I open it after the last update", "inbound": True},
        {"text": "The wifi keeps disconnecting on my new phone every few minutes", "inbound": True},
        {"text": "Getting an error message when trying to install the latest version", "inbound": True},
        {"text": "My device is running extremely slow since yesterday's update", "inbound": True},
        {"text": "The screen keeps freezing and I have to reboot my phone", "inbound": True},
        {"text": "Bluetooth connection drops every time I walk away from my desk", "inbound": True},
        {"text": "Software update failed and now my device won't turn on", "inbound": True},
        {"text": "There's a major bug in the latest release causing data loss", "inbound": True},
        {"text": "My printer stopped working after the driver update", "inbound": True},
        {"text": "The iOS app crashes on startup every single time now", "inbound": True},
        {"text": "Can't download anything, keeps showing connection error", "inbound": True},
        {"text": "Android app has a glitch where notifications don't show up", "inbound": True},
        {"text": "My internet connection is extremely slow and keeps lagging", "inbound": True},
        {"text": "The sync feature is broken and won't update across devices", "inbound": True},
        {"text": "Battery drains very fast since the latest software update", "inbound": True},
        {"text": "Hardware issue with the screen display showing weird colors", "inbound": True},
        {"text": "App keeps loading forever and never actually opens properly", "inbound": True},
        {"text": "My device keeps restarting on its own randomly throughout the day", "inbound": True},
        {"text": "Fix the lag issue please it makes the app unusable", "inbound": True},
        {"text": "The disconnect problem happens every time I use mobile data", "inbound": True},

        # ── Billing (20 samples) ──
        {"text": "I was charged twice on my credit card for the same order!", "inbound": True},
        {"text": "Can I get a refund for my last month's subscription?", "inbound": True},
        {"text": "The payment didn't go through but I was still charged", "inbound": True},
        {"text": "I want to cancel my subscription and get my money back", "inbound": True},
        {"text": "Why did the price increase without any notification?", "inbound": True},
        {"text": "I need a copy of my invoice from last month please", "inbound": True},
        {"text": "The subscription fee was charged even after I cancelled", "inbound": True},
        {"text": "My credit card shows an overcharge of twenty dollars", "inbound": True},
        {"text": "How do I renew my plan at the current pricing?", "inbound": True},
        {"text": "I need a receipt for the payment I made yesterday", "inbound": True},
        {"text": "Is there a discount or coupon code available for annual plans?", "inbound": True},
        {"text": "The cost of the premium plan seems way too high", "inbound": True},
        {"text": "I made a payment but it doesn't show up in my billing history", "inbound": True},
        {"text": "Please refund the incorrect charge on my account immediately", "inbound": True},
        {"text": "My bill shows charges for services I never signed up for", "inbound": True},
        {"text": "How do I update my payment method to a new credit card?", "inbound": True},
        {"text": "The subscription renewal charged me at a higher price than expected", "inbound": True},
        {"text": "I need to cancel my plan and stop future charges", "inbound": True},
        {"text": "Why am I being charged a fee for something that should be free?", "inbound": True},
        {"text": "The pricing page shows a different cost than what I was charged", "inbound": True},

        # ── Account (20 samples) ──
        {"text": "I can't log into my account, it says my password is wrong", "inbound": True},
        {"text": "How do I reset my password? I forgot it completely", "inbound": True},
        {"text": "My account got locked after too many login attempts", "inbound": True},
        {"text": "I need to update my email address on my profile", "inbound": True},
        {"text": "Can't access my account, the verification code isn't coming through", "inbound": True},
        {"text": "How do I change my username? I want a different one", "inbound": True},
        {"text": "I want to register a new account but it keeps giving errors", "inbound": True},
        {"text": "My account security settings need to be updated", "inbound": True},
        {"text": "I can't sign in with my existing credentials anymore", "inbound": True},
        {"text": "Please help me verify my email address for the account", "inbound": True},
        {"text": "I need to set up two factor authentication on my account", "inbound": True},
        {"text": "My profile picture won't upload or change in my settings", "inbound": True},
        {"text": "Someone may have access to my account without my permission", "inbound": True},
        {"text": "How do I sign up for a new account using my phone number?", "inbound": True},
        {"text": "I need to reset my account password urgently", "inbound": True},
        {"text": "The login page keeps timing out when I try to sign in", "inbound": True},
        {"text": "Can you send me a DM with my account recovery details?", "inbound": True},
        {"text": "My account email verification link has expired", "inbound": True},
        {"text": "I want to delete my account and all associated data", "inbound": True},
        {"text": "How do I log in with my social media account instead?", "inbound": True},
    ]
    return pd.DataFrame(data)


def load_and_label(path: str, sample_size: int = SAMPLE_SIZE) -> pd.DataFrame:
    """Load the CSV, keep inbound customer tweets, and assign labels.
    Falls back to built-in sample if file not found."""
    if os.path.exists(path):
        print(f"📂 Loading data from {path}...")
        df = pd.read_csv(path)
        df = df[df["inbound"] == True].copy()
        df = df.dropna(subset=["text"])
        if len(df) > sample_size:
            df = df.sample(n=sample_size, random_state=RANDOM_STATE)
            print(f"   Sampled {sample_size} inbound messages for speed.")
    else:
        print(f"⚠️  File '{path}' not found. Using built-in sample dataset (60 messages).")
        df = create_sample_dataset()

    # Assign labels
    df["category"] = df["text"].apply(assign_label)
    df = df.dropna(subset=["category"])

    # Ensure each category has at least 2 samples
    counts = df["category"].value_counts()
    valid = counts[counts >= 2].index
    df = df[df["category"].isin(valid)]

    return df.reset_index(drop=True)


print("✅ Data loading functions defined.")

In [ ]:
# Load the data
df = load_and_label(DATA_PATH)

print(f"\n📊 Dataset size: {len(df)} labeled messages")
print(f"\nLabel distribution:")
print(df["category"].value_counts().to_string())
print(f"\n--- Sample rows ---")
df[["text", "category"]].head(10)

### 4.1 Data Distribution Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart of label distribution
category_counts = df["category"].value_counts()
colors = ["#4A90D9", "#E8875B", "#5CB85C"]
category_counts.plot(kind="bar", ax=axes[0], color=colors, edgecolor="white")
axes[0].set_title("Category Distribution", fontsize=14, fontweight="bold")
axes[0].set_ylabel("Count")
axes[0].set_xlabel("")
axes[0].tick_params(axis="x", rotation=0)

# Pie chart
category_counts.plot(kind="pie", ax=axes[1], colors=colors, autopct="%1.1f%%",
                      startangle=90, textprops={"fontsize": 11})
axes[1].set_title("Category Proportions", fontsize=14, fontweight="bold")
axes[1].set_ylabel("")

plt.tight_layout()
plt.show()

## 5. Text Preprocessing

In [ ]:
# Apply preprocessing to create clean text column
df["clean_text"] = df["text"].apply(preprocess_text)

# Show before/after comparison
print("Before vs After Preprocessing:")
print("=" * 80)
for i in range(min(5, len(df))):
    print(f"  Raw:     {df['text'].iloc[i][:75]}")
    print(f"  Clean:   {df['clean_text'].iloc[i][:75]}")
    print(f"  Label:   {df['category'].iloc[i]}")
    print("-" * 80)

# Basic text stats
df["word_count"] = df["clean_text"].apply(lambda x: len(x.split()))
print(f"\n📏 Average word count per message: {df['word_count'].mean():.1f}")
print(f"   Min: {df['word_count'].min()}, Max: {df['word_count'].max()}")

## 6. Build & Train Model

In [ ]:
def build_pipeline() -> Pipeline:
    """Create a TF-IDF + Logistic Regression pipeline."""
    return Pipeline([
        ("tfidf", TfidfVectorizer(
            max_features=5000,
            ngram_range=(1, 2),
            stop_words="english",
        )),
        ("clf", LogisticRegression(
            max_iter=1000,
            solver="lbfgs",
            random_state=RANDOM_STATE,
        )),
    ])


print("✅ Pipeline builder defined.")

In [ ]:
# Split the data
X = df["clean_text"]
y = df["category"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y,
)

print(f"🔀 Train/Test Split:")
print(f"   Training set: {len(X_train)} samples")
print(f"   Test set:     {len(X_test)} samples")
print(f"\n   Training label distribution:")
print(f"   {y_train.value_counts().to_dict()}")
print(f"\n   Test label distribution:")
print(f"   {y_test.value_counts().to_dict()}")

In [ ]:
# Build and train the pipeline
pipeline = build_pipeline()
pipeline.fit(X_train, y_train)

print("✅ Model trained successfully!")
print(f"   TF-IDF vocabulary size: {len(pipeline.named_steps['tfidf'].vocabulary_)}")

## 7. Model Evaluation

In [ ]:
# Predictions
y_pred = pipeline.predict(X_test)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"🎯 Overall Accuracy: {acc:.2%}")
print()

# Classification Report
print("=" * 55)
print("           CLASSIFICATION REPORT")
print("=" * 55)
print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
# Confusion Matrix - Visual
labels = sorted(y.unique())
cm = confusion_matrix(y_test, y_pred, labels=labels)

fig, ax = plt.subplots(figsize=(7, 5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(ax=ax, cmap="Blues", values_format="d")
ax.set_title("Confusion Matrix", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

# Also as a DataFrame for clarity
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
cm_df.index.name = "Actual"
cm_df.columns.name = "Predicted"
print("\nConfusion Matrix (DataFrame):")
cm_df

### 7.1 Cross-Validation

In [ ]:
# 5-fold cross-validation for more robust evaluation
cv_pipeline = build_pipeline()
cv_scores = cross_val_score(cv_pipeline, X, y, cv=5, scoring="accuracy")

print(f"📈 5-Fold Cross-Validation Results:")
print(f"   Fold scores: {[f'{s:.2%}' for s in cv_scores]}")
print(f"   Mean accuracy: {cv_scores.mean():.2%} (± {cv_scores.std():.2%})")

### 7.2 Top TF-IDF Features per Category

In [ ]:
# Show the most important words for each category
tfidf = pipeline.named_steps["tfidf"]
clf = pipeline.named_steps["clf"]
feature_names = np.array(tfidf.get_feature_names_out())

print("🔑 Top 10 TF-IDF Features per Category:")
print("=" * 55)

for i, label in enumerate(clf.classes_):
    top_indices = clf.coef_[i].argsort()[-10:][::-1]
    top_features = feature_names[top_indices]
    top_weights = clf.coef_[i][top_indices]
    print(f"\n  [{label}]")
    for feat, weight in zip(top_features, top_weights):
        bar = "█" * int(abs(weight) * 5)
        print(f"    {feat:20s} {weight:+.3f}  {bar}")

## 8. Prediction Function & Demo

In [ ]:
def predict_category(pipeline: Pipeline, text: str) -> dict:
    """Classify a single support message and return prediction + probabilities."""
    cleaned = preprocess_text(text)
    prediction = pipeline.predict([cleaned])[0]
    probabilities = pipeline.predict_proba([cleaned])[0]
    prob_dict = dict(zip(pipeline.classes_, probabilities))
    return {
        "text": text,
        "predicted_category": prediction,
        "confidence": max(probabilities),
        "probabilities": prob_dict,
    }


print("✅ Prediction function defined.")

In [ ]:
# ── Demo Predictions ──
demo_messages = [
    "I was charged twice on my credit card for the same order!",
    "My app keeps crashing every time I open it after the update.",
    "I can't log into my account, it says my password is wrong.",
    "The wifi keeps disconnecting on my new phone.",
    "Can I get a refund for my last month's subscription?",
    "How do I reset my password? I forgot it.",
    "My internet is extremely slow since yesterday.",
    "I need to update my billing address on my profile.",
]

print("=" * 65)
print("                    DEMO PREDICTIONS")
print("=" * 65)

results = []
for msg in demo_messages:
    result = predict_category(pipeline, msg)
    cat = result["predicted_category"]
    conf = result["confidence"]
    print(f"  [{cat:>10}] ({conf:.0%})  {msg}")
    results.append(result)

print()

In [ ]:
# Visualize prediction probabilities for demo messages
fig, axes = plt.subplots(2, 4, figsize=(16, 6))
axes = axes.flatten()

for i, result in enumerate(results):
    probs = result["probabilities"]
    cats = list(probs.keys())
    vals = list(probs.values())
    bar_colors = ["#4A90D9" if c == result["predicted_category"] else "#D0D0D0" for c in cats]

    axes[i].barh(cats, vals, color=bar_colors, edgecolor="white")
    axes[i].set_xlim(0, 1)
    axes[i].set_title(f"Msg {i+1}: {result['predicted_category']}", fontsize=10, fontweight="bold")
    axes[i].tick_params(axis="y", labelsize=9)

plt.suptitle("Prediction Confidence per Demo Message", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

## 9. Interactive Testing

Run the cell below and type your own support messages to classify them in real time.

In [ ]:
# ── Interactive classifier ──
# Type a message and press Enter. Type 'quit' to stop.

print("=" * 55)
print("    🎫 Interactive Support Ticket Classifier")
print("    Type a message below (or 'quit' to exit)")
print("=" * 55)

while True:
    try:
        user_input = input("\n📝 Your message: ")
    except (EOFError, KeyboardInterrupt):
        print("\n👋 Goodbye!")
        break

    if user_input.strip().lower() in ("quit", "exit", "q"):
        print("👋 Goodbye!")
        break

    if not user_input.strip():
        print("   ⚠️  Please enter a message.")
        continue

    result = predict_category(pipeline, user_input)
    print(f"   🏷️  Category: {result['predicted_category']}")
    print(f"   📊 Confidence: {result['confidence']:.1%}")
    for cat, prob in sorted(result['probabilities'].items(), key=lambda x: -x[1]):
        bar = "█" * int(prob * 30)
        print(f"      {cat:>10}: {prob:.1%} {bar}")

## 10. Summary & Model Export (Optional)

In [ ]:
import joblib

# Save the trained pipeline
model_path = "support_ticket_classifier.pkl"
joblib.dump(pipeline, model_path)
print(f"💾 Model saved to: {model_path}")

# Summary
print()
print("=" * 55)
print("              📋 PROJECT SUMMARY")
print("=" * 55)
print(f"  Dataset size:      {len(df)} labeled messages")
print(f"  Categories:        {', '.join(sorted(y.unique()))}")
print(f"  Model:             TF-IDF + Logistic Regression")
print(f"  Test accuracy:     {acc:.2%}")
print(f"  CV mean accuracy:  {cv_scores.mean():.2%} (± {cv_scores.std():.2%})")
print(f"  Vocabulary size:   {len(pipeline.named_steps['tfidf'].vocabulary_)}")
print(f"  Model saved to:    {model_path}")
print("=" * 55)

In [ ]:
# Verify saved model loads correctly
loaded_pipeline = joblib.load(model_path)
test_msg = "I can't login to my account"
result = loaded_pipeline.predict([preprocess_text(test_msg)])[0]
print(f"✅ Loaded model verification: '{test_msg}' → {result}")